# `import logging`

Going beyond 

```python
logging.basicConfig(level="INFO"); logging.info("Hello")
```

---

**Table of contents**

[TOC]

In [1]:
import logging
import logging.config
import importlib
import sys

def reload() -> None:
    """Call this at every cell to reset logging config"""
    importlib.reload(logging)
    importlib.reload(logging.config)

# Logging flow

*[Link to docs](https://docs.python.org/3/howto/logging.html#logging-flow)*

![](https://docs.python.org/3/_images/logging_flow.png)

## Objects we need to understand
1. Loggers
2. Handler
3. Formatter
4. Filters
5. Log Record

In [2]:
# How to give a code example of this?

# Loggers
## Naming loggers

Loggers apply the first level of filtering and emit the log record.

To get loggers, we use the `logging.getLogger(__name__)` helper function. 
Using the module  (`__name__`) as logger name is common.

In [3]:
reload()

logger = logging.getLogger("foo")

print(logger, logger.name, type(logger), sep="\n")

<Logger foo (WARNING)>
foo
<class 'logging.Logger'>


## Loggers parents and childs

Loggers are named with herarchical system using dots (`.`).

In [4]:
reload()

logger_parent = logging.getLogger("foo")

logger_child = logging.getLogger("foo.bar")

print(f"The parent of {logger_child=} is {logger.parent=}")

The parent of logger_child=<Logger foo.bar (WARNING)> is logger.parent=<RootLogger root (WARNING)>


Child loggers will propagate (i.e. forward) log records to their parents if:
- The child is enabled for 
- the `propagate` attribute is set to `True`.

**Pseudo code**
```python
## PSEUDO
# logger: Current Logger
# log_record: log record being emited

logger.emit(log_record)
while "logger has parent" and logger.propagate:
    parent_name = logger.name.split(".")[0]
    logger = logging.getLogger(parent_name)

    if logger.disabled or "logger is not enabled for log_record":
        break

    logger.emit(log_record)
```

In [5]:
reload()

logger_parent = logging.getLogger("foo")

logger_child = logging.getLogger("foo.bar")

logger_parent.setLevel(logging.INFO)
logger_child.setLevel(logging.INFO)

# we will go into more depth into handlers and formatters latter
# this will print 
# parent messages with the format ---message----
# child messages with the format /\/\message/\/\

handler_parent = logging.StreamHandler(sys.stdout)
handler_parent.formatter = logging.Formatter("---%(msg)s---")
logger_parent.addHandler(handler_parent)

handler_child = logging.StreamHandler(sys.stdout)
handler_child.formatter = logging.Formatter("/\\/\\%(msg)s/\\/\\")
logger_child.addHandler(handler_child)

# With propagate, the message is emited by both loggers
logger_child.propagate = True
logger_child.info("Hello")

# Without propagate, the child will never pass the message to the parent
logger_child.propagate = False
logger_child.info("Goodbye")

/\/\Hello/\/\
---Hello---
/\/\Goodbye/\/\


# The Root Logger

The root logger is the parent of all loggers that don't have a dot in their name.

I.e. the root logger is the ancestor of every other logger.

In [6]:
reload()

root_logger = logging.getLogger() # or logging.root
child_of_root = root_logger.getChild("foo")
named_top_level_logger = logging.getLogger("foo")

print("Root logger:", root_logger)
print(f"Q: Is the '{root_logger}' the parent of logger '{named_top_level_logger}'?\nA: {named_top_level_logger.parent is root_logger}")
print(f"Q: Is '{child_of_root=}' the same as '{named_top_level_logger=}'?\nA: {child_of_root is named_top_level_logger}")


Root logger: <RootLogger root (WARNING)>
Q: Is the '<RootLogger root (WARNING)>' the parent of logger '<Logger foo (WARNING)>'?
A: True
Q: Is 'child_of_root=<Logger foo (WARNING)>' the same as 'named_top_level_logger=<Logger foo (WARNING)>'?
A: True


## Loggers: Level

In [7]:
reload()

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

# If the set log level is less or equal than the message
# the log record is emitted 
logger.setLevel(logging.INFO)
logger.info("Hello")

# If the set log level is greather than the message
# the log record is ignored 
logger.setLevel(logging.WARNING)
logger.info("Goodbye")


INFO:foo:Hello


## Loggers: Disabled

In [8]:
reload()

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

logger.disabled = False
logger.info("Hello")

logger.disabled = True
logger.info("Goodbye")

INFO:foo:Hello


## Custom logger classes

You can extend and modify the behaviour of the Logger class.

Some example use cases are:
- Json loggers
- Context loggers
- etc

In [9]:
reload()

class MyNoisyLogger(logging.Logger):
    def makeRecord(self, *args, **kwargs) -> logging.LogRecord:
        print("I'm making a log record")
        return super().makeRecord(*args, *kwargs)

logging.basicConfig(level="INFO")

logging.setLoggerClass(MyNoisyLogger)

logger = logging.getLogger("foo")
logger.info("Hello")

INFO:foo:Hello


I'm making a log record


# Handlers
Handlers act upon log records do perform an specific action.

Some example actions:
- Print to terminal
- Append to file
- Send a HTTP request
- etc

Handlers are also responsible for formatting the log record and transforming them to strings. See [Formatters](#formatters).

In [10]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

# without any handlers
logger.info("Hello")

# with a StreamHandler
logger.addHandler(logging.StreamHandler(sys.stdout))
print(logger.handlers)
logger.info("Staying")

# with another StreamHandler
# the message gets handled by each handler independantly
logger.addHandler(logging.StreamHandler(sys.stdout))
print(logger.handlers)
logger.info("Goodbye")

[<StreamHandler stdout (NOTSET)>]
Staying
[<StreamHandler stdout (NOTSET)>, <StreamHandler stdout (NOTSET)>]
Goodbye
Goodbye


## Handlers: levels and and other attributes
Handlers, like loggers, have a level and [filters](#filters).

Contrary to loggers:
- they are not hierarchical and don't propagate anything
- they do not have a enabled/disabled state

In [11]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

handler: logging.Handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)

handler.setLevel(logging.INFO)
print(logger.handlers)
logger.info("Hello")

handler.setLevel(logging.WARNING)
print(logger.handlers)
logger.info("Hello")  # gets ignored by the handler

[<StreamHandler stdout (INFO)>]
Hello
[<StreamHandler stdout (WARNING)>]


## Other important handlers

## Useful Handlers

*[Link to docs](https://docs.python.org/3/howto/logging.html#useful-handlers)*

Python has a lot of useful handlers already included in the `logging.handlers` module.

# Formatters

# Filters

Filters are functions or classes that are called upon a LogRecord to determine if it should be filtered.

In [12]:
reload()

def filter_ignore_me(record: logging.LogRecord) -> bool:
    """True means the message should be preserved"""
    return "IGNORED" not in record.msg

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")
logger.info("Hello")
logger.info("This message should be IGNORED but there is no filter")

logger.addFilter(filter_ignore_me)
logger.info("This message should also be IGNORED")
logger.info("Goodbye")

INFO:foo:Hello
INFO:foo:This message should be IGNORED but there is no filter
INFO:foo:Goodbye


In [13]:
reload()

class MyFilter(logging.Filter):
    """A Filter can also be an instance of logging.Filter"""

    def filter(self, record: logging.LogRecord) -> bool:
        """A filter may also modify inplace the Log Record"""
        if "IGNORED" in record.msg:
            return False
        if "IMPORTANT" in record.msg:
            # I wouldn't do this in real life
            record.levelname = "WARNING"
            record.levelno = 30
            record.msg = record.msg.replace("IMPORTANT", "").strip()
        return True

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

logger.addFilter(MyFilter())
logger.info("Hello")
logger.info("This message should be IGNORED")
logger.info("IMPORTANT Goodbye")

INFO:foo:Hello


## Filter in both Loggers and Handlers

# Tips and FAQ
1. Numbered log levels
2. Why late formatting
3. Logging in libraries: NullHandler and LastResort.
4. `extra`